In [ ]:
import os
import datetime

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

# **Data Loading**

In [ ]:
df = pd.read_csv('../input/jena-climate-2009-2016/jena_climate_2009_2016.csv', index_col=None)

display(df.head())
display(df.tail())

In [ ]:
df.shape

# **Data Analysis**

In [ ]:
df.describe().transpose()

In [ ]:
corr = df.corr()
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0,
                 cmap=sns.diverging_palette(20, 220, n=200), square=True)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
# indices of point of split
print(df[df["Date Time"]=='31.12.2014 23:50:00'].index.values)
print(df[df["Date Time"]=='31.12.2015 23:50:00'].index.values)

# **Data Manipulation**

## **Replace default empty value**

In [ ]:
for col in ['wv (m/s)', 'max. wv (m/s)']:
    df[col] = df[col].replace(-9999.00, 0)

## **Removing the redundant features**

In [ ]:
features = [list(df.columns)[c] for c in [0,1,2,6,8,9,11,12]]
features

In [ ]:
df_rel = df[features].copy()
df_rel.head()

## **Sin-Cos Extraction for Date-Time column**

In [ ]:
day = 24*60*60
year = (365.2425)*day

date_time = pd.to_datetime(df_rel.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')
t = date_time.map(datetime.datetime.timestamp)
t_day = t * (2*np.pi/day)
t_year = t * (2*np.pi/year)
df_rel['Day_sin'] = np.sin(t_day)
df_rel['Day_cos'] = np.cos(t_day)
df_rel['Year_sin'] = np.sin(t_year)
df_rel['Year_cos'] = np.cos(t_year)

df_rel.head()

## **Split 6-year training data**

In [ ]:
split_fraction = 0.75075

train_split = int(split_fraction * int(df_rel.shape[0]))

# Apply normalization on ALL data, but using only TRAINING data
data_mean = df_rel[:train_split].mean(axis=0)
data_std = df_rel[:train_split].std(axis=0)
df_rel = (df_rel-data_mean) / data_std

In [ ]:
train_data = df_rel.loc[:train_split-1]
val_data = df_rel.loc[train_split:]

display(train_data.head())
display(val_data.head())

# **Modeling**

## **Data Loader**

In [ ]:
from keras.preprocessing import timeseries_dataset_from_array

In [ ]:
step = 6
past = 720
future = 72
patience = 3
batch_size = 256

In [ ]:
start = past + future
end = start + train_split

x_train = train_data.values
y_train = df_rel.iloc[start:end]['T (degC)']
display(y_train.head())

sequence_length = int(past / step)

In [ ]:
print('X_train shape ==', x_train.shape)
print('y_train shape ==', y_train.shape)

In [ ]:
dataset_train = timeseries_dataset_from_array(x_train, y_train,
                                              sequence_length=sequence_length,
                                              sampling_rate=step,
                                              batch_size=batch_size,)

for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

In [ ]:
x_end = len(val_data) - past - future
y_start = train_split + past + future

x_val = val_data.iloc[:x_end].values
y_val = df_rel.iloc[y_start:]['T (degC)']

dataset_val = timeseries_dataset_from_array(x_val, y_val,
                                            sequence_length=sequence_length,
                                            sampling_rate=step,
                                            batch_size=batch_size,)

## **Build model**

In [ ]:
pip install gradient-centralization-tf

In [ ]:
import gctf

from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

In [ ]:
inputs = Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = LSTM(units=48)(inputs)
outputs = Dense(units=1, activation='linear')(lstm_out)

model = Model(inputs=inputs, outputs=outputs, name='LSTM')
model.compile(optimizer=gctf.optimizers.adam(learning_rate=1e-3), loss="mse")
model.summary()

## **Training**

In [ ]:
%%time
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopper = EarlyStopping(monitor="val_loss", min_delta=0, patience=patience*2+1, restore_best_weights=True,)
checkpointer = ModelCheckpoint(monitor="val_loss", filepath="model_checkpoint.h5", verbose=1, save_weights_only=True, save_best_only=True)
lr_reducer = ReduceLROnPlateau(monitor="val_loss", factor=0.169, patience=patience, verbose=1)

history = model.fit(x=dataset_train, 
                    epochs=50,
                    validation_data=dataset_val,
                    callbacks=[early_stopper, checkpointer, lr_reducer])

In [ ]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

visualize_loss(history, "Training and Validation Loss")

## **Evaluation**

In [ ]:
model.load_weights('./model_checkpoint.h5')

std = data_std[1]
avg = data_mean[1]
   
def show_plot(plot_data, future: int=0, title: str='Title'):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return

for x, y in dataset_val.take(20):
    p = x[0][:, 1].numpy()
    q = y[0].numpy()
    p = p*std + avg
    q = q*std + avg
    pred = model.predict(x)[0]
    pred = (pred*std + avg)
    show_plot([p, q, pred], future=12, title="Single Step Prediction")